In [2]:
import pandas as pd
import pandoc
import csv
import os
import re
import pickle
import numpy as np
from tqdm import tqdm
import spacy
from spacy.pipeline import EntityRuler
from nltk import Tree
import json

FileNotFoundError: [Errno 2] No such file or directory: '/Users/nb/PycharmProjects/pythonProject/venv3/lib/python3.8/site-packages/pandoc/definitions/1.16.hs'

In [10]:
base_nlp = spacy.load('de_core_news_lg')

### Options

In [ ]:
debug = False
use_lsk = False
n_splits = 10

output_path = r"C:\Users\natal\Desktop\Korpus\debug_1812"
output_path = r'D:\Natalie\13_korpus\run_bote_210216'
#output_path = r'D:\Natalie\13_korpus\run_mag_210216'

### Gazetteer Files

In [12]:
gazetteer_LIN = r"D:\Natalie\07_gazetteer\swissNames3D_LIN.csv"
gazetteer_PKT = r"D:\Natalie\07_gazetteer\swissNames3D_PKT.csv"
gazetteer_PLY = r"D:\Natalie\07_gazetteer\swissNames3D_PLY.csv"

### Text Files

In [13]:
text_files_eval= r"F:\Korpus\pages_Bote"
#text_files_eval= r"F:\Korpus\pages_Magazin"
#text_files_eval= r"C:\Users\natal\Desktop\Korpus\test_pages"
text_lsk= r"F:\Korpus\LSK_Text"

korpus_data = text_files_eval

if use_lsk:
    korpus_data = text_lsk

### Loading

In [14]:
abbreviations = [
    ("Nr.", "Nr"),
    ("z.B.", "zB"),
    ("zB.", "zB"), 
    ("St.", "St")
]

sentences = []
for ordner in os.listdir(korpus_data):
    for text_file in tqdm(os.listdir(os.path.join(korpus_data, ordner))):
        with open(os.path.join(korpus_data, ordner, text_file), encoding='utf-8') as file:
            data = file.read()
            for abbr in abbreviations:
                data = data.replace(abbr[0], abbr[1])
            file_sentences = re.split(r' *[\.\?!] *', data) # orig: split nach Punkten
            sentences += file_sentences
    if debug:
        #break
        pass
        
if debug:
    sentences = sentences[:1000]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2888/2888 [00:26<00:00, 108.20it/s]


In [15]:
print(len(sentences)) #alle Sätze: 5'522'722
print(type(sentences))

5443296
<class 'list'>


In [ ]:
#sentences

In [ ]:
def gaz_to_df(gaz):
    df = pd.read_csv(gaz, sep=';', encoding='utf-8', usecols=["NAME", "UUID", "OBJEKTART", "POINT_X", "POINT_Y", "POINT_Z", "Type"])
    #print(df.columns)
    return df

gazetteer_LIN_df = gaz_to_df(gazetteer_LIN)
gazetteer_PKT_df = gaz_to_df(gazetteer_PKT)
gazetteer_PLY_df = gaz_to_df(gazetteer_PLY)
gazetteer_all = pd.concat([gazetteer_LIN_df, gazetteer_PKT_df, gazetteer_PLY_df])


In [ ]:
gazetteer_all

In [ ]:
def perform_ner(X):
    entities=[]
    sentences=[]
    #perform named entity recognition using spacy
    for line in tqdm(X):
        text_ner = base_nlp(line)
        temp_ents = []
        for ent in text_ner.ents:
            if ent.label_ == "LOC":
                temp_ents.append(ent.text)
                if len(sentences)==0 or sentences[-1] != line:
                    sentences.append(line)
        if temp_ents:
            entities.append(temp_ents)
    #falls die entity als Location identifiziert wird, soll der dazugehörige Satz gespeichert werden.
    return entities, sentences

In [ ]:
def search_gazetteer_rules(X, gazetteer):
    sentences_winfo=[]
    sentences=[]
    toponym_nr_ids={}
    toponym_count={}
    topo_id_count={}
    topo_in_sen=[]
    
    endings = ["er", "in", "innen", "Innen","s", "en","ers", "erisch","isch","e", "n", "ig", "es", "ieren", "eln", "ung"]
    pattern = "nm4you"
    for uuid, topo, objektart, typus, coord_x, coord_y, coord_z in zip(gazetteer.iloc[:,0], gazetteer.iloc[:,2], gazetteer.iloc[:,1], gazetteer.iloc[:,6], gazetteer.iloc[:,3], gazetteer.iloc[:,4], gazetteer.iloc[:,5]):    
        if topo not in toponym_nr_ids:
            toponym_nr_ids[topo] = [{"uuid": uuid, "objektart": objektart, "geom": typus, "coords": (coord_x, coord_y, coord_z)}]
            pattern += f"|{topo}(\\b"
            for ending in endings:
                pattern += f"|{ending}\\b"
            pattern += ")"   
        else:
            toponym_nr_ids[topo].append({"uuid": uuid, "objektart": objektart, "geom": typus, "coords": (coord_x, coord_y, coord_z)})

    for sen in tqdm(X):
        document = re.sub(r'\W', ' ', str(sen))
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)  
        document = re.sub(r'\^[a-zA-Z]\s+', ' ', document)  
        document = re.sub(r'\s+', ' ', document, flags=re.I) 
        document = re.sub(r'^b\s+', '', document) 
        sen = document
        
        results = list(re.finditer(pattern, sen))
        
        if results:
            sentences.append(sen)
            topo_list = []
            info_dict = {"sentence": sen, "topo_infos": []}
            for res in results:
                result = res.string[res.start():res.end()]
                topo_list.append(result)
                for topo in toponym_nr_ids:
                    if any([topo + ending == result for ending in endings + [""]]):
                        new_topo_info = {"topo": topo, "info": toponym_nr_ids[topo]}
                        if not new_topo_info in info_dict["topo_infos"]:
                            info_dict["topo_infos"].append(new_topo_info)
                        if topo not in topo_id_count:
                            topo_id_count[topo]=len(toponym_nr_ids[topo])
                if result not in toponym_count:
                    toponym_count[result] = 1
                else:
                    toponym_count[result] += 1
            topo_in_sen.append(topo_list)
            sentences_winfo.append(info_dict)
  
    return sentences, toponym_nr_ids, toponym_count, topo_id_count, topo_in_sen, sentences_winfo

### Save Topo Sentences to PKL

In [ ]:
splitsize = int(len(sentences)/n_splits)+1

os.makedirs(output_path, exist_ok=True)

for i in range(n_splits):
    nc_split = sentences[i*splitsize:(i+1)*splitsize]
    #nc_split = nc_split[:10]
    print("perfom ner ", i)
    entities, saetze_ner = perform_ner(X=nc_split)
    print("perform regex ", i)
    saetze, toponym_ids, toponym_counts, topo_id_counts, topo_in_sen, sentences_winfo = search_gazetteer_rules(X=saetze_ner, gazetteer=gazetteer_all)
    
    pickle_data = {
        "entities":entities , 
        "saetze_ner":saetze_ner, 
        "saetze":saetze, 
        "toponym_ids":toponym_ids, 
        "toponym_counts":toponym_counts, 
        "topo_id_counts":topo_id_counts, 
        "topo_in_sen":topo_in_sen, 
        "sentences_winfo":sentences_winfo
    }
    with open(f"{output_path}/temp_topo_{i}.pkl", 'wb') as file:
        pickle.dump(pickle_data, file)

### Load <sentences_winfo> again from PKL

In [ ]:
all_topos_in_corpus=[]

for temp_topo_file in os.listdir(output_path):
    if temp_topo_file.startswith("temp_topo"):
        with open(os.path.join(output_path, temp_topo_file), 'rb') as fileo:
            print(os.path.join(output_path, temp_topo_file))
            data_frompickle = pickle.load(fileo)
            all_topos_in_corpus.append(data_frompickle['sentences_winfo'])
        

In [ ]:
all_topos_in_corpus

### Process Topos

In [ ]:
UUID = []
TOPO = []
for row in all_topos_in_corpus:
    for sen in row:
        for topo_info in sen['topo_infos']:
            for info in topo_info['info']:
                UUID.append(info['uuid'])
                TOPO.append(topo_info['topo'])
                break


with open(f"{output_path}/all_topos_in_corpus.csv", 'w', encoding="utf-8") as output_file:
    csv_out=csv.writer(output_file)
    csv_out.writerow(['topo','uuid'])
    for topo, uuid in zip(TOPO, UUID):
        csv_out.writerow([topo, uuid]) 































In [ ]:
#sorted(toponym_counts.items())
print(len(toponym_counts))
print(len(toponym_ids))
print(len(topo_id_counts))
print(len(saetze))
print(len(topo_in_sen))

In [ ]:
topo_id_counts.items()

# After Finding LSB

# Nicht mehr brauchen, wegen Simons tollem Algorithmus im combine

### Toponym NER --> LSB --> Toponym mit UUID dranspeichern

In [ ]:
## für ein File, das schon pro Linie einen Satz enthält

eval_pre=[]
debug_info=[]
classifications=[]
saetze=[]

for file in os.listdir(output_path):
    if file.startswith('temp_lsb'):
        with open(os.path.join(output_path, file), 'rb') as fileo:
            print(os.path.join(output_path, file))
            data = pickle.load(fileo)
            eval_pre += data["eval_pre"]
            debug_info += data["debug_info"]
            classifications.append(data["eval_pred"])
        if debug:
            break
    
classifications = np.concatenate(classifications)

for res, sen, debinfo in sorted(zip(classifications, eval_pre, debug_info)):
        if res == 1:
            saetze.append(sen+"\n")
            
all_saetze, all_toponym_ids, all_toponym_counts, all_topo_id_counts, all_topo_in_sen, all_sentences_winfo = search_gazetteer_rules(X=saetze, gazetteer=gazetteer_all)

In [ ]:
# open a file where data need to be stored
file = open(f"{output_path}/temp_both.pkl", 'wb')
# dump information to the file
pickle.dump(all_sentences_winfo, file)
# close the file
file.close()
print(file)
          

In [ ]:
print(len(sentences_winfo))

In [ ]:
ambig = 0
unique = 0
topo_id_counts_ambig=[]
for topo, id_count in topo_id_counts.items():
    print(topo)
    print(id_count)
    if id_count > 1:
        ambig += 1
        topo_id_counts_ambig.append(topo)
    else:
        unique += 1
print(ambig)
print(unique)
print(len(topo_id_counts))
print(ambig/len(topo_id_counts))
print(unique/len(topo_id_counts))

In [ ]:
ambig_1topo=0
ambig_mehretopo=0

for topo, sentence in sorted(zip(topo_in_sen, saetze)):
    if len(topo) > 1:
        ambig_mehretopo += 1
        for t in topo:
            if t in topo_id_counts_ambig:
                print("Mehrere Topo: ", t, ": \n",  sentence) 
    else:
        ambig_1topo += 1
        for t in topo:
            if t in topo_id_counts_ambig:
                print("Nur 1 Topo: ", t, ": \n", sentence)
print(ambig_1topo)
print(ambig_mehretopo)
print(ambig_1topo/(ambig_1topo+ambig_mehretopo))

In [ ]:
sorted(topo_id_counts.items())
print(sorted(topo_id_counts.items(), key=lambda kv: kv[1]))
#print(saetze)

# Precision & Recall - Vergleich zweier Methoden

In [ ]:
text_files_orig = r"C:\Users\natal\Desktop\Studium\01 Masterarbeit\08_Evaluation\original_2Round"

text_files_annot = r"C:\Users\natal\Desktop\Studium\01 Masterarbeit\08_Evaluation\annotiert_2Round"

In [ ]:
no_context = []
for text_file in tqdm(os.listdir(text_files_annot)):
    with open(os.path.join(text_files_annot, text_file), encoding='utf-8') as file:
        data = file.read()
        #no_context.append('\n'.join(file.readlines()))
        sentences = re.split(r' *[\.\?!][\'"\)\]]* *', data)
        no_context += sentences

In [ ]:
def load_annot(X, marker="xxx"):
    marked_list = []
    for line in X:
        for word in line.replace("\n", " ").split(" "):
            if marker in word:
                for part in word.split(marker):
                    part_stripped = part.rstrip("-\n/?)“„«,.(:")
                    if part_stripped:
                        marked_list.append(part_stripped)
    return marked_list

In [ ]:
def count_topos(marked_list):
    topo_dict = {}
    for sublist in marked_list:
        for topo in sublist:
            if topo not in topo_dict:
                topo_dict[topo] = 1
            else:
                topo_dict[topo] += 1
    return topo_dict

In [ ]:
#### Annotierte Docs
marked_list = load_annot(X=no_context, marker="xxx")
ground_truth_dict = count_topos(marked_list)

print(len(ground_truth_dict))
for k in sorted(ground_truth_dict.keys()):
    print(k, ground_truth_dict[k])

In [ ]:
#### Originale Docs - NER und Gazetter mit Rules
no_context = []
for text_file in tqdm(os.listdir(text_files_orig)):
    with open(os.path.join(text_files_orig, text_file), encoding='utf-8') as file:
        data = file.read()
        #no_context.append('\n'.join(file.readlines()))
        sentences = re.split(r' *[\.\?!][\'"\)\]]* *', data)
        no_context += sentences


In [ ]:
_, ner_loc_list_temp = perform_ner(X=no_context)
saetze, toponym_ids, toponym_counts, topo_id_counts, ner_loc_list = search_gazetteer_rules(X=ner_loc_list_temp, gazetteer=gazetteer_all)

In [ ]:
for topo, satz in zip(ner_loc_list, saetze):
    print(topo, " in ", satz, "-----------------------")

In [ ]:
topos_count_dict = count_topos(ner_loc_list)

print(len(topos_count_dict))
for k in sorted(topos_count_dict.keys()):
    print(k, topos_count_dict[k], ",")


In [ ]:
#### Originale Docs - nur Gazetteer mit Rules, ohne NER
   
saetze, toponym_ids, toponym_counts, topo_id_counts, gaz_loc_list = search_gazetteer_rules(X=no_context, gazetteer=gazetteer_all)

topos_count_dict = count_topos(gaz_loc_list)

print(len(topos_count_dict))
for k in sorted(topos_count_dict.keys()):
    print(k, topos_count_dict[k], ",")

In [ ]:
#### Precision: wieviele Topos sind relevant von allen Resultaten
### True Positives / Trues Positives + False Positives
def precision(results, ground_truth):
    true_p = 0
    false_p = 0
    for topo in ground_truth:
        if topo in results:
            true_p = true_p + min(ground_truth[topo], results[topo])
    for topo in results:
        if topo not in ground_truth:
            false_p += results[topo]
        else: 
            false_p += min(results[topo] - ground_truth[topo], 0)
    return true_p/(true_p + false_p)

#### Recall: wieviele Topos, die ich brauchen kann, sind in den Resultaten
### True Positives / True Positives + False Negatives
def recall(results, ground_truth):
    true_p = 0
    for topo in ground_truth:
        if topo in results:
            true_p = true_p + min(ground_truth[topo], results[topo])
    return true_p/sum(ground_truth.values())

print(precision(topos_count_dict, ground_truth_dict))
print(recall(topos_count_dict, ground_truth_dict))

In [ ]:
# NER & Gazetteer
0.7362110311750599
0.6303901437371663

# Nur Gazetteer
0.568760611205433
0.6878850102669405

In [ ]:
s, t, tc, tic, tis, res = search_gazetteer_rules(X=["Hallo  Alpthal Birchli, Birchli, Birchli, Hoch Ybrig und Hoch-Ybrig"], gazetteer=gazetteer_all)
res
with open(r"C:\Users\natal\Desktop\test.json", "w") as write_file:
    json.dump(res, write_file, indent=4)